In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import glob
import cv2

In [3]:
# image 로드(read and resize)
def load_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img = img.astype(np.float32)
    img = (img - np.mean(img)) / np.std(img)
    return img


In [4]:
# data loading from file
def load_data():
    load_image_counting=0
    dataset_dir = '/content/drive/MyDrive/DS2020/annotation'
    image_paths = glob.glob('{}/images/*.jpg'.format(dataset_dir))
    image_paths.sort()

    train_images = []
    train_labels = np.empty(shape=[0,2])
    test_images = []
    test_labels = np.empty(shape=[0,2])

    train_list = open('{}/annotations/trainval.txt'.format(dataset_dir), 'r')
    test_list = open('{}/annotations/test.txt'.format(dataset_dir), 'r')

    cat = np.array([[1,0]])
    dog = np.array([[0,1]])

    Lines = train_list.readlines()
    for line in Lines:
        load_image_counting+=1
        print('image_loading',load_image_counting)
        train_images.append(load_image(os.path.join(dataset_dir, 'images', line.split()[0] + '.jpg')))
        if (float(line.split()[2])==1):               # 1일때는 cat nparray append
          train_labels=np.append(train_labels,cat,axis=0)
        else:                                         # 0일때는 dog nparray append
          train_labels=np.append(train_labels,dog,axis=0)

    Lines2 = test_list.readlines()
    for line in Lines2:
        load_image_counting+=1
        print('image_loading',load_image_counting)
        test_images.append(load_image(os.path.join(dataset_dir, 'images', line.split()[0] + '.jpg')))
        if (float(line.split()[2])==1):               # 1일때는 cat nparray append
          test_labels=np.append(test_labels,cat,axis=0)
        else:                                         # 0일때는 dog  nparray append
          test_labels=np.append(test_labels,dog,axis=0)

    train_images = np.array(train_images)
    test_images = np.array(test_images)

    return train_images, train_labels, test_images, test_labels



In [5]:
def build_and_compile_model():
    model = keras.Sequential([
        keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same', input_shape=(128, 128, 3)),
        keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
        keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'),
        keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
        keras.layers.Conv2D(128, kernel_size=3, activation='relu', padding='same'),
        keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation=tf.nn.relu), keras.layers.Dropout(0.2),
        keras.layers.Dense(256, activation=tf.nn.relu), keras.layers.Dropout(0.2),
        keras.layers.Dense(2, activation=tf.nn.sigmoid)
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])         # cat, dog binary 문제이기 때문에 binary_crossentropy 사용
    model.summary()
    return model

In [6]:
train_features, train_labels, test_features, test_labels = load_data()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
image_loading 2350
image_loading 2351
image_loading 2352
image_loading 2353
image_loading 2354
image_loading 2355
image_loading 2356
image_loading 2357
image_loading 2358
image_loading 2359
image_loading 2360
image_loading 2361
image_loading 2362
image_loading 2363
image_loading 2364
image_loading 2365
image_loading 2366
image_loading 2367
image_loading 2368
image_loading 2369
image_loading 2370
image_loading 2371
image_loading 2372
image_loading 2373
image_loading 2374
image_loading 2375
image_loading 2376
image_loading 2377
image_loading 2378
image_loading 2379
image_loading 2380
image_loading 2381
image_loading 2382
image_loading 2383
image_loading 2384
image_loading 2385
image_loading 2386
image_loading 2387
image_loading 2388
image_loading 2389
image_loading 2390
image_loading 2391
image_loading 2392
image_loading 2393
image_loading 2394
image_loading 2395
image_loading 2396
image_loading 2397
image_loading 2398
image_loading 2399
image_loading 

In [7]:
cnn_model = build_and_compile_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0

In [8]:
cnn_model.fit(train_features, train_labels, epochs=50, validation_split=0.2, verbose=1, shuffle=True)

Epoch 1/50
92/92 [==============================] - 10s 25ms/step - loss: 0.7865 - accuracy: 0.6592 - val_loss: 0.6180 - val_accuracy: 0.6630
Epoch 2/50
92/92 [==============================] - 2s 21ms/step - loss: 0.5845 - accuracy: 0.6748 - val_loss: 0.6140 - val_accuracy: 0.6671
Epoch 3/50
92/92 [==============================] - 2s 21ms/step - loss: 0.5405 - accuracy: 0.7301 - val_loss: 0.5945 - val_accuracy: 0.6685
Epoch 4/50
92/92 [==============================] - 2s 21ms/step - loss: 0.4878 - accuracy: 0.7727 - val_loss: 0.5922 - val_accuracy: 0.7215
Epoch 5/50
92/92 [==============================] - 2s 21ms/step - loss: 0.3918 - accuracy: 0.8353 - val_loss: 0.6330 - val_accuracy: 0.7201
Epoch 6/50
92/92 [==============================] - 2s 21ms/step - loss: 0.2640 - accuracy: 0.8963 - val_loss: 0.7822 - val_accuracy: 0.7283
Epoch 7/50
92/92 [==============================] - 2s 21ms/step - loss: 0.1522 - accuracy: 0.9468 - val_loss: 0.9698 - val_accuracy: 0.7255
Epoch 8/50
9

In [9]:
test_loss, test_acc = cnn_model.evaluate(test_features, test_labels,verbose=1)

115/115 [==============================] - 1s 8ms/step - loss: 1.8850 - accuracy: 0.7427


In [11]:
print('\n\ntest accuracy:', test_acc)
test_predictions = cnn_model.predict(test_features).flatten()



test accuracy: 0.7427091598510742
